# Analýza věku a pohlaví podle pětiletých věkových skupin

### Import knihoven


In [1]:
import pandas as pd
import numpy as np
import pymongo
import utils.data_loader as data_loader
import pprint
import json
from cis import *

### Poznámky

In [2]:
# https://www.czso.cz/csu/rso/uzemne_technicke_jednotky
# https://scitani.gov.cz/vysledky
# https://www.czso.cz/csu/czso/vysledky-scitani-2021-otevrena-data

### Pomocné funkce

In [2]:
# Tisk čísla s oddělovači čárek
def format_with_commas(number):
    return "{:,}".format(number)

# Hezký tisk slovníku pro věk
def print_dictionary_ages(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}: {value} ({(value / analysis_result['count'])*100:.2f}%)")

# Hezký tisk slovníku pro věk a pohlaví
def print_dictionary_ages_sex(text, dict, total):
    print(f"{text}")
    for outer_key, outer_value in dict.items():
        print(f"\t=> {outer_key} ({(dict[outer_key]['total'] / total) * 100:.2f} %):")
        for key, value in outer_value.items():
            print(f"\t\t=> {key}: {value} ({(value / dict[outer_key]['total']) * 100:.2f} %)")

### Uložení CSV dat do MongoDB

In [3]:
data_loader.load_csv_into_db("vek_pohlavi_5", "../data_original/sldb2021_vek5_pohlavi.csv")

KeyboardInterrupt: 

### Načtení dat do paměti z MongoDB

In [4]:
data = data_loader.load_from_db("obyvatelstvo_pohlavi_vek_5")

### Analýza

In [3]:
# Pomocné CIS kódy
cis_vek = {
    1100000004: '0 - 4 roky',
    1300050009: '5 - 9 let',
    1300100014: '10 - 14 let',
    1300150019: '15 - 19 let',
    1300200024: '20 - 24 let',
    1300250029: '25 - 29 let',
    1300300034: '30 - 34 let',
    1300350039: '35 - 39 let',
    1300400044: '40 - 44 let',
    1300450049: '45 - 49 let',
    1300500054: '50 - 54 let',
    1300550059: '55 - 59 let',
    1300600064: '60 - 64 let',
    1300650069: '65 - 69 let',
    1300700074: '70 - 74 let',
    1300750079: '75 - 79 let',
    1300800084: '80 - 84 let',
    1300850089: '85 - 89 let',
    1300900094: '90 - 94 let',
    1300950099: '95 - 99 let',
    1201009999: '100 a více let'
}

vek_txt = {
    '0 - 4 roky': 1,
    '5 - 9 let': 2,
    '10 - 14 let': 3,
    '15 - 19 let': 4,
    '20 - 24 let': 5,
    '25 - 29 let': 6,
    '30 - 34 let': 7,
    '35 - 39 let': 8,
    '40 - 44 let': 9,
    '45 - 49 let': 10,
    '50 - 54 let': 11,
    '55 - 59 let': 12,
    '60 - 64 let': 13,
    '65 - 69 let': 14,
    '70 - 74 let': 15,
    '75 - 79 let': 16,
    '80 - 84 let': 17,
    '85 - 89 let': 18,
    '90 - 94 let': 19,
    '95 - 99 let': 20,
    '100 a více let': 21 
}

vzdelani_txt = {
    'Bez vzdělání': 1,
    'Neúplné základní vzdělání': 2,
    'Základní vzdělání': 3,
    'Úplné střední všeobecné vzdělání': 8,
    'Vyšší odborné vzdělání': 11,
    'Vysokoškolské bakalářské vzdělání': 12,
    'Vysokoškolské magisterské vzdělání': 13,
    'Vysokoškolské doktorské vzdělání': 14,
    'Vyšší odborné vzdělání v konzervatoři': 15,
    'Nižší střední a střední vzdělání': 105,
    'Úplné střední odborné vzdělání': 121,
    'Nástavbové vzdělání': 202,
    'Pomaturitní studium': 203,
    'Nezjištěno': 900
}

cis_kraje = {
    3026: 'Středočeský kraj',
    3018: 'Hlavní město Praha',
    3115: 'Jihomoravský kraj',
    3140: 'Moravskoslezský kraj',
    3069: 'Ústecký kraj',
    3034: 'Jihočeský kraj',
    3123: 'Olomoucký kraj',
    3042: 'Plzeňský kraj',
    3131: 'Zlínský kraj',
    3085: 'Královéhradecký kraj',
    3093: 'Pardubický kraj',
    3107: 'Kraj Vysočina',
    3077: 'Liberecký kraj',
    3051: 'Karlovarský kraj'
}

veky_podle_fazi = {
    'deti': ['0 - 4 roky', '5 - 9 let'],
    'dospivani': ['10 - 14 let','15 - 19 let'],
    'dospeli': ['20 - 24 let', '25 - 29 let', '30 - 34 let', '35 - 39 let', '40 - 44 let', '45 - 49 let', '50 - 54 let', '55 - 59 let'],
    'stari': ['60 - 64 let', '65 - 69 let', '70 - 74 let', '75 - 79 let', '80 - 84 let', '85 - 89 let', '90 - 94 let', '95 - 99 let', '100 a více let'],
}

In [12]:
analysis_result = dict()

# Celkový počet obyvatel
analysis_result['count'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle pohlaví
analysis_result['count_women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 2)]['hodnota'])
analysis_result['count_men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věku
analysis_result['count_ages'] = {}
for key, value in cis_vek.items():
    analysis_result['count_ages'][value] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věku a pohlaví
analysis_result['count_age_sex'] = {}
for key, value in cis_vek.items():
    analysis_result['count_age_sex'][value] = {}
    analysis_result['count_age_sex'][value]['women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 2)]['hodnota'])
    analysis_result['count_age_sex'][value]['men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období)
analysis_result['count_age_phase'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase'][key] = 0
    for phase in value:
        analysis_result['count_age_phase'][key] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období) a pohlaví
analysis_result['count_age_phase_sex'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase_sex'][key] = {}
    analysis_result['count_age_phase_sex'][key]['women'] = 0
    analysis_result['count_age_phase_sex'][key]['men'] = 0
    for phase in value:
        analysis_result['count_age_phase_sex'][key]['women'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 2)]['hodnota'])
        analysis_result['count_age_phase_sex'][key]['men'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle okresu, věku a pohlaví
analysis_result['count_sex_district'] = {}
for key, value in cis_okresy.items():
    key_int = int(key)
    analysis_result['count_sex_district'][key_int] = {}
    analysis_result['count_sex_district'][key_int]['name'] = value
    analysis_result['count_sex_district'][key_int]['data'] = {}
    analysis_result['count_sex_district'][key_int]['data']['women'] = []
    analysis_result['count_sex_district'][key_int]['data']['men'] = []
    for key2, value2 in cis_vek.items(): 
        analysis_result['count_sex_district'][key_int]['data']['women'].append(int(data[(data['uzemi_kod'] == key_int) & (data['uzemi_cis'] == 101) & (data['pohlavi_kod'] == 2) & (data['vek_txt'] == value2)]['hodnota']))
        analysis_result['count_sex_district'][key_int]['data']['men'].append(int(data[(data['uzemi_kod'] == key_int) & (data['uzemi_cis'] == 101) & (data['pohlavi_kod'] == 1) & (data['vek_txt'] == value2)]['hodnota']))

# Výsledky analýzy

In [13]:
# Celkový počet obyvatel
print(f"Celkový počet obyvatel: {format_with_commas(analysis_result['count'])}")
print(f"\nCelkový počet žen: {format_with_commas(analysis_result['count_women'])} ({(analysis_result['count_women'] / analysis_result['count'])*100:.2f}%)")
print(f"\nCelkový počet muž: {format_with_commas(analysis_result['count_men'])} ({(analysis_result['count_men'] / analysis_result['count'])*100:.2f}%)")
print_dictionary_ages("\nCelkový počet obyvatel podle věku:", analysis_result['count_ages'])
print_dictionary_ages_sex("\nCelkový počet obyvatel podle věku a pohlaví:", analysis_result['count_age_sex'])
print_dictionary_ages("\nCelkový počet obyvatel podle věkoých fází:", analysis_result['count_age_phase'])
print_dictionary_ages_sex("\nCelkový počet obyvatel podle věkových fází a pohlaví:", analysis_result['count_age_phase_sex'])
# print_dictionary_ages_sex("\nCelkový počet obyvatel podle pohlaví a kraje:", analysis_result['count_sex_district'])
# pprint.pprint(analysis_result['count_sex_district'])
# output = json.dumps(analysis_result['count_sex_district'], ensure_ascii=False).encode('utf8')
# with open('output.json', 'w') as f:
#     f.write(output.decode())

Celkový počet obyvatel: 10,524,167

Celkový počet žen: 5,337,619 (50.72%)

Celkový počet muž: 5,186,548 (49.28%)

Celkový počet obyvatel podle věku:
	=> 0 - 4 roky: 560632 (5.33%)
	=> 5 - 9 let: 550024 (5.23%)
	=> 10 - 14 let: 581104 (5.52%)
	=> 15 - 19 let: 489836 (4.65%)
	=> 20 - 24 let: 477063 (4.53%)
	=> 25 - 29 let: 601168 (5.71%)
	=> 30 - 34 let: 690561 (6.56%)
	=> 35 - 39 let: 719089 (6.83%)
	=> 40 - 44 let: 857917 (8.15%)
	=> 45 - 49 let: 877616 (8.34%)
	=> 50 - 54 let: 688110 (6.54%)
	=> 55 - 59 let: 666391 (6.33%)
	=> 60 - 64 let: 616608 (5.86%)
	=> 65 - 69 let: 668081 (6.35%)
	=> 70 - 74 let: 618629 (5.88%)
	=> 75 - 79 let: 419368 (3.98%)
	=> 80 - 84 let: 243373 (2.31%)
	=> 85 - 89 let: 135311 (1.29%)
	=> 90 - 94 let: 52786 (0.50%)
	=> 95 - 99 let: 9853 (0.09%)
	=> 100 a více let: 647 (0.01%)

Celkový počet obyvatel podle věku a pohlaví:
	=> 0 - 4 roky:
		=> women: 273743
		=> men: 286889
	=> 5 - 9 let:
		=> women: 267952
		=> men: 282072
	=> 10 - 14 let:
		=> women: 283743


In [30]:
import pandas as pd

data_path = '../data_original/sldb2021_vek5_pohlavi.csv'
data = pd.read_csv(data_path)
data_filtered = data[data['vek_txt'].notna() & data['pohlavi_txt'].isna()]
data_filtered = data_filtered.loc[data_filtered['uzemi_cis'] == 97]
data_filtered['vek_txt_ciselny'] = data_filtered['vek_txt'].map(vek_txt)
data_filtered.head()


,idhod,hodnota,ukaz_kod,vek_cis,vek_kod,pohlavi_cis,pohlavi_kod,uzemi_cis,uzemi_kod,sldb_rok,sldb_datum,ukaz_txt,vek_txt,pohlavi_txt,uzemi_txt,vek_txt_ciselny
437187,964588155,560632,3162,1035.0,1.100000e+09,NaN,NaN,97,19,2021,2021-03-26,Počet obyvatel s obvyklým pobytem,0 - 4 roky,NaN,Česká republika,0
437190,964588148,550024,3162,1035.0,1.300050e+09,NaN,NaN,97,19,2021,2021-03-26,Počet obyvatel s obvyklým pobytem,5 - 9 let,NaN,Česká republika,1
437193,964588730,581104,3162,1035.0,1.300100e+09,NaN,NaN,97,19,2021,2021-03-26,Počet obyvatel s obvyklým pobytem,10 - 14 let,NaN,Česká republika,2
437196,964588147,489836,3162,1035.0,1.300150e+09,NaN,NaN,97,19,2021,2021-03-26,Počet obyvatel s obvyklým pobytem,15 - 19 let,NaN,Česká republika,3
437199,964588146,477063,3162,1035.0,1.300200e+09,NaN,NaN,97,19,2021,2021-03-26,Počet obyvatel s obvyklým pobytem,20 - 24 let,NaN,Česká republika,4


In [27]:
# Definování věkových skupin a jejich počtů
age_groups = {
    "0 - 4 roky": 560632,
    "5 - 9 let": 550024,
    "10 - 14 let": 581104,
    "15 - 19 let": 489836,
    "20 - 24 let": 477063,
    "25 - 29 let": 601168,
    "30 - 34 let": 690561,
    "35 - 39 let": 719089,
    "40 - 44 let": 857917,
    "45 - 49 let": 877616,
    "50 - 54 let": 688110,
    "55 - 59 let": 666391,
    "60 - 64 let": 616608,
    "65 - 69 let": 668081,
    "70 - 74 let": 618629,
    "75 - 79 let": 419368,
    "80 - 84 let": 243373,
    "85 - 89 let": 135311,
    "90 - 94 let": 52786,
    "95 - 99 let": 9853,
    "100 a více let": 647,
}

# Oprava extrakce minimálního a maximálního věku
weighted_age_sum_corrected = 0

# Upravení logiky pro extrakci čísel z řetězce
for age_range, count in age_groups.items():
    if age_range == "100 a více let":
        representative_age = 102  # Speciální případ pro "100 a více let"
    else:
        # Upravení pro extrakci číselného rozsahu z řetězců
        min_age, max_age = [int(x) for x in age_range.replace(' let', '').replace(' roky', '').split(' - ')]
        representative_age = (min_age + max_age) / 2
    
    weighted_age_sum_corrected += representative_age * count

average_age_corrected = weighted_age_sum_corrected / 10524167
average_age_corrected


42.20571271816572

In [62]:
# Poměr mezi muži a ženy v obcích

import json

f = open('data/obce_kod_nazev.json')
obce_kod_nazev = json.load(f)['obce_kod_nazev']

f = open('data/obce_kod_pocet.json')
obce_kod_pocet = json.load(f)['obce_kod_pocet']

data_filtered_obce = data[data['vek_txt'].isna() & data['pohlavi_txt'].notna()]
data_filtered_obce = data_filtered_obce.loc[data_filtered_obce['uzemi_cis'] == 43]

# Poměr pohlaví
best = 100
best_obec = 0
for obec in obce_kod_nazev:
    muzi = data_filtered_obce.loc[(data_filtered_obce['uzemi_kod'] == int(obec)) & (data_filtered_obce['pohlavi_kod'] == 1)]['hodnota'].values[0]
    zeny = data_filtered_obce.loc[(data_filtered_obce['uzemi_kod'] == int(obec)) & (data_filtered_obce['pohlavi_kod'] == 2)]['hodnota'].values[0]
    pomer = (muzi / zeny) * 100
    if pomer < best:
        best = pomer
        best_obec = obec

print(best)
print(best_obec)

50.0
541061


- nejvíce dětí v poměru s celkovým počtem
- počet obyvatel na km2

In [65]:
# JSON - rozloha

import pandas as pd
import json

df = pd.read_excel('data/obce_rozloha.xlsx')
obce_kod_rozloha = {}

for index, row in df.iterrows():
    kod = row['kod']
    rozloha = row['rozloha'] / 100
    obce_kod_rozloha[kod] = rozloha

data_to_save = {'obce_kod_rozloha': obce_kod_rozloha}

with open('data/obce_kod_rozloha.json', 'w') as json_file:
    json.dump(data_to_save, json_file, ensure_ascii=False, indent=4)


In [68]:
# Hustota zalidneni

f = open('data/obce_kod_nazev.json')
obce_kod_nazev = json.load(f)['obce_kod_nazev']

f = open('data/obce_kod_pocet.json')
obce_kod_pocet = json.load(f)['obce_kod_pocet']

f = open('data/obce_kod_rozloha.json')
obce_kod_rozloha = json.load(f)['obce_kod_rozloha']

min = 99999999
min_obec = None
max = 0
max_obec = None

for obec in obce_kod_nazev:
    hustota = obce_kod_pocet[obec] / obce_kod_rozloha[obec]
    if hustota > max:
        max = hustota
        max_obec = obec
    if hustota < min:
        min = hustota
        min_obec = obec

print('Min: ' + str(min) + ' (' + obce_kod_nazev[min_obec] + ')')
print('Max: ' + str(max) + ' (' + obce_kod_nazev[max_obec] + ')')

Min: 1.0901417914356404 (Modrava)
Max: 2622.923102014264 (Praha)


In [75]:
# Nejlepsi pomery deti a starsich v obcich

deti_max = 0
deti_max_obec = None
deti_min = 999999999
deti_min_obec = None
stari_max = 0
stari_max_obec = None
stari_min = 99999999
stari_min_obec = None

for obec in obce_kod_nazev:
    pocet_deti = 0
    pocet_stari = 0
    pocet_celkem = obce_kod_pocet[obec]
    obec_stats = data[(data['uzemi_cis'] == 43) & (data['uzemi_kod'] == int(obec)) & (data['pohlavi_kod'].isna())]
    for deti_skup in veky_podle_fazi['deti']:
        pocet_deti += obec_stats[(obec_stats['vek_txt'] == deti_skup)]['hodnota'].values[0]
    for stari_skup in veky_podle_fazi['stari']:
        pocet_stari += obec_stats[(obec_stats['vek_txt'] == stari_skup)]['hodnota'].values[0]
    
    procento_deti = (pocet_deti / pocet_celkem) * 100
    procento_stari = (pocet_stari / pocet_celkem) * 100

    if procento_deti > deti_max:
        deti_max = procento_deti
        deti_max_obec = obec
    if procento_deti < deti_min:
        deti_min = procento_deti
        deti_min_obec = obec
    if procento_stari > stari_max:
        stari_max = procento_stari
        stari_max_obec = obec
    if procento_stari < stari_min:
        stari_min = procento_stari
        stari_min_obec = obec

print('Max pomer deti: ' + str(deti_max) + ' (' + obce_kod_nazev[deti_max_obec] + ')')
print('Min pomer deti: ' + str(deti_min) + ' (' + obce_kod_nazev[deti_min_obec] + ')')
print('Max pomer stari: ' + str(stari_max) + ' (' + obce_kod_nazev[stari_max_obec] + ')')
print('Min pomer stari: ' + str(stari_min) + ' (' + obce_kod_nazev[stari_min_obec] + ')')

Max pomer deti: 25.0 (Horní Myslová)
Min pomer deti: 0.0 (Čejkovice)
Max pomer stari: 81.25 (Vysoká Lhota)
Min pomer stari: 6.148055207026349 (Lesní Jakubov)


In [5]:
data_loader.load_csv_into_db("vzdelani_vek_pohlavi", "../data_original/sldb2021_vzdelani_vek2_pohlavi_reduced.csv")

In [5]:
data = data_loader.load_from_db("vzdelani_vek_pohlavi")
data.head()

,_id,idhod,hodnota,vzdelani_cis,vzdelani_kod,vek_kod,pohlavi_cis,pohlavi_kod,uzemi_cis,uzemi_kod,vzdelani_txt,vek_txt,pohlavi_txt,uzemi_txt,uzemi_typ
0,66058d38135f8b5ec087c225,1065348324,6,1294,1,1300150064,None,None,43,500011,Bez vzdělání,15 - 64 let,None,Želechovice nad Dřevnicí,obec
1,66058d38135f8b5ec087c226,1065987611,4,1294,1,1300150064,102.0,1.0,43,500011,Bez vzdělání,15 - 64 let,muž,Želechovice nad Dřevnicí,obec
2,66058d38135f8b5ec087c227,1065439653,2,1294,1,1300150064,102.0,2.0,43,500011,Bez vzdělání,15 - 64 let,žena,Želechovice nad Dřevnicí,obec
3,66058d38135f8b5ec087c228,1066034432,3,1294,1,1200659999,None,None,43,500011,Bez vzdělání,65 a více let,None,Želechovice nad Dřevnicí,obec
4,66058d38135f8b5ec087c229,1065233243,2,1294,1,1200659999,102.0,1.0,43,500011,Bez vzdělání,65 a více let,muž,Želechovice nad Dřevnicí,obec


In [25]:
# Vzdelani a jejich pomer oproti celku

analysis_result = {}
analysis_result['count_education_sex'] = {}
count = 0

for key, value in vzdelani_txt.items():
    analysis_result['count_education_sex'][key] = {}
    analysis_result['count_education_sex'][key]['women'] = data[
        (data['uzemi_cis'] == 97) & 
        (data['vzdelani_kod'] == value) & 
        (data['vek_kod'].isin([1300150064, 1200659999])) & 
        (data['pohlavi_kod'] == 2)
    ]['hodnota'].sum()
    analysis_result['count_education_sex'][key]['men'] = data[
        (data['uzemi_cis'] == 97) & 
        (data['vzdelani_kod'] == value) & 
        (data['vek_kod'].isin([1300150064, 1200659999])) & 
        (data['pohlavi_kod'] == 1)
    ]['hodnota'].sum()
    analysis_result['count_education_sex'][key]['total'] = analysis_result['count_education_sex'][key]['men'] + analysis_result['count_education_sex'][key]['women']
    count += analysis_result['count_education_sex'][key]['total']

In [26]:
print_dictionary_ages_sex("\nCelkový počet obyvatel podle vzdělání a pohlaví:", analysis_result['count_education_sex'], total=count)


Celkový počet obyvatel podle vzdělání a pohlaví:
	=> Bez vzdělání (0.64 %):
		=> women: 27440 (48.91 %)
		=> men: 28660 (51.09 %)
		=> total: 56100 (100.00 %)
	=> Neúplné základní vzdělání (1.27 %):
		=> women: 59781 (53.16 %)
		=> men: 52684 (46.84 %)
		=> total: 112465 (100.00 %)
	=> Základní vzdělání (11.27 %):
		=> women: 604955 (60.78 %)
		=> men: 390440 (39.22 %)
		=> total: 995395 (100.00 %)
	=> Úplné střední všeobecné vzdělání (12.70 %):
		=> women: 676678 (60.34 %)
		=> men: 444829 (39.66 %)
		=> total: 1121507 (100.00 %)
	=> Vyšší odborné vzdělání (1.43 %):
		=> women: 84747 (67.26 %)
		=> men: 41243 (32.74 %)
		=> total: 125990 (100.00 %)
	=> Vysokoškolské bakalářské vzdělání (3.75 %):
		=> women: 200884 (60.69 %)
		=> men: 130105 (39.31 %)
		=> total: 330989 (100.00 %)
	=> Vysokoškolské magisterské vzdělání (12.96 %):
		=> women: 593633 (51.88 %)
		=> men: 550699 (48.12 %)
		=> total: 1144332 (100.00 %)
	=> Vysokoškolské doktorské vzdělání (0.87 %):
		=> women: 28679 (37.2

In [10]:
# Nejlepsi pomery ve vzdelani v okresech

max = 0
max_okres = None
min = 999999999
min_okres = None

for okres in cis_okresy:
    pocet_vys = 0

    okres_stats = data[(data['uzemi_cis'] == 101) & (data['uzemi_kod'] == int(okres)) & (data['pohlavi_kod'].isna()) & (data['vek_txt'].isin(['15 - 64 let', '65 a více let']))]
    pocet_celkem = okres_stats['hodnota'].sum()

    for vys_skup in vzdelani_podle_typu['bez_vzdelani']:
        pocet_vys += okres_stats[(okres_stats['vzdelani_txt'] == vys_skup)]['hodnota'].sum()
    
    procento_vys = (pocet_vys / pocet_celkem) * 100

    if procento_vys > max:
        max = procento_vys
        max_okres = okres
    if procento_vys < min:
        min = procento_vys
        min_okres = okres

print('Max pomer vysokoskolsky vzdelanych v okresech: ' + str(max) + ' (' + cis_okresy[max_okres] + ')')
print('Min pomer vysokoskolsky vzdelanych v okresech: ' + str(min) + ' (' + cis_okresy[min_okres] + ')')

Max pomer vysokoskolsky vzdelanych v okresech: 1.517942921729302 (Most)
Min pomer vysokoskolsky vzdelanych v okresech: 0.35533358800721915 (Praha)


In [18]:
# Nejlepsi pomery ve vzdelani v obcich (alespon 2000 obyvatel)

f = open('data/obce_kod_nazev.json')
obce_kod_nazev = json.load(f)['obce_kod_nazev']

f = open('data/obce_kod_pocet.json')
obce_kod_pocet = json.load(f)['obce_kod_pocet']

max = 0
max_obec = None
min = 999999999
min_obec = None

for obec in obce_kod_nazev:
    pocet_vys = 0
    pocet_celkem = obce_kod_pocet[obec]

    # Preskocit obce pod 2000 obyvatel
    if pocet_celkem < 3000: continue

    obec_stats = data[(data['uzemi_cis'] == 43) & (data['uzemi_kod'] == int(obec)) & (data['pohlavi_kod'].isna()) & (data['vek_txt'].isin(['15 - 64 let', '65 a více let']))]

    for vys_skup in vzdelani_podle_typu['vysokoskolske_vzdelani']:
        pocet_vys += obec_stats[(obec_stats['vzdelani_txt'] == vys_skup)]['hodnota'].sum()
    
    procento_vys = (pocet_vys / pocet_celkem) * 100

    if procento_vys > max:
        max = procento_vys
        max_obec = obec
    if procento_vys < min:
        min = procento_vys
        min_obec = obec

print('Max pomer vysokoskolsky vzdelanych v obcich nad 3000: ' + str(max) + ' (' + obce_kod_nazev[max_obec] + ' - ' + max_obec + ')')
print('Min pomer vysokoskolsky vzdelanych v obcich nad 3000: ' + str(min) + ' (' + obce_kod_nazev[min_obec] + ' - ' + min_obec + ')')

Max pomer vysokoskolsky vzdelanych v obcich nad 3000: 29.8819255222525 (Černošice - 539139)
Min pomer vysokoskolsky vzdelanych v obcich nad 3000: 3.7572254335260116 (Jiříkov - 562581)


In [17]:
# Nejlepsi pomery ve vzdelani v krajich

max = 0
max_kraj = None
min = 999999999
min_kraj = None

for kraj in cis_kraje:
    pocet_vys = 0

    kraj_stats = data[(data['uzemi_cis'] == 100) & (data['uzemi_kod'] == int(kraj)) & (data['pohlavi_kod'].isna()) & (data['vek_txt'].isin(['15 - 64 let', '65 a více let']))]
    pocet_celkem = kraj_stats['hodnota'].sum()

    for vys_skup in vzdelani_podle_typu['vysokoskolske_vzdelani']:
        pocet_vys += kraj_stats[(kraj_stats['vzdelani_txt'] == vys_skup)]['hodnota'].sum()
    
    procento_vys = (pocet_vys / pocet_celkem) * 100

    if procento_vys > max:
        max = procento_vys
        max_kraj = kraj
    if procento_vys < min:
        min = procento_vys
        min_kraj = kraj

print('Max pomer vysokoskolsky vzdelanych v krajich: ' + str(max) + ' (' + cis_kraje[max_kraj] + ')')
print('Min pomer vysokoskolsky vzdelanych v krajich: ' + str(min) + ' (' + cis_kraje[min_kraj] + ')')

Max pomer vysokoskolsky vzdelanych v krajich: 33.69598652708602 (Hlavní město Praha)
Min pomer vysokoskolsky vzdelanych v krajich: 9.641167442509419 (Karlovarský kraj)
